## Read in config file and do basic checks.

In [ ]:
report_problem()
{
    date | tee -a /tmp/debuglog
    echo "$*" | tee -a /tmp/debuglog
    exit 1 # exit the bash kernel (which does not exit nbconvert :-( ))
}

source ~/jh_test.config || report_problem "problem reading config file"

check_ssh()
{
    [ "$(echo $SSHCMD)" != "" ] || report_problem "SSHCMD is not set"
    [ "$($SSHCMD echo OK)" = "OK" ] || report_problem "SSHCMD ($SSHCMD) did not connect successfully"
}

check_targetdir()
{
    $SSHCMD "[ -d '$TARGETDIR' ]" || report_problem "The directory ($TARGETDIR) does not exist on the server"
}

check_bridge()
{
    [ "$(echo $BRIDGENAME)" != "" ] || report_problem "BRIDGENAME is not set"
    [[ "$($SSHCMD /sbin/brctl show "$BRIDGENAME" 2>&1 1>/dev/null)" == "" ]] || report_problem "bridge ($BRIDGENAME) not found"
}

check_ssh
check_targetdir
check_bridge
echo "Config checks OK"

## Run the next cell to copy the build repository to the target directory.


In [ ]:
$SSHCMD rm -fr "'$TARGETDIR/installdir'"

if $SSHCMD [ -d "$TARGETDIR/jupyter-platform-dev" ]; then
    echo "Repository already copied.  Its status is as follows:"
    $SSHCMD <<EOF
        set -e
        cd "$TARGETDIR/jupyter-platform-dev"
        git status
EOF
else
    echo "Copying the repository..."
    cd ~/
    tar c jupyter-platform-dev/.git | $SSHCMD tar x -C "$TARGETDIR"
    $SSHCMD <<EOF
        set -e
        cd "$TARGETDIR/jupyter-platform-dev"
        git reset --hard
        echo "Repository status is as follows:"
        git status
EOF
fi

## Run next cell to set up the install directory:
This cell will run quickly, finishing within a few seconds.

In [ ]:
script="$TARGETDIR/jupyter-platform-dev/ind-steps/build-jh-environment/toplevel-kvm-tap-build.sh-new"

$SSHCMD <<EOF
PATH=/sbin:$PATH
"$script" "$TARGETDIR/installdir" "$BRIDGENAME"
EOF

## Run the next cell to do the actual build.
It should take about 90 minutes to run.

In [ ]:
# $SSHCMD time "$TARGETDIR/installdir/toplevel-kvm-tap-build.sh" do

## Run the next cell to check the status of each build step.
It should take about 20 seconds to run.

In [ ]:
$SSHCMD "$TARGETDIR/installdir/toplevel-kvm-tap-build.sh" check